In [120]:
#TSKファジィ推論を示す
import numpy as np
import pandas as pd
import random
import math
import sys

In [121]:
#まず医療診断データを読み取る
diabetes_data = pd.read_csv('fuzzy_diabetes2_1.csv')

In [122]:
#教師データの数は73、テストデータの数は72と設定する
TEACHER_SIZE = 73
TEST_SIZE = 72

In [123]:
#教師データとテストデータに分割する
teacher_data = diabetes_data.iloc[0:TEACHER_SIZE]
test_data = diabetes_data.iloc[TEACHER_SIZE:]

In [124]:
#説明変数と被説明変数に分ける
x_teacher_data = np.array(teacher_data[['input1','input2','input3','input4','input5']])
y_teacher_data = np.array(teacher_data['output'])
x_test_data = np.array(test_data[['input1','input2','input3','input4','input5']])
y_test_data = np.array(test_data['output'])

In [125]:
#ファジィルールの数と、ファジィ分割数、入力変数の数を設定する
NUMBER_OF_FUZZY_PARTITION = 3
NUMBER_OF_INPUT = 5
NUMBER_OF_FUZZY_RULE = NUMBER_OF_FUZZY_PARTITION ** NUMBER_OF_INPUT

In [126]:
#前件部の初期値と後件部の初期値を設定する
#前件部の中心と幅は、ルール数×各ルールにおける入力変数の数
antecedent_center = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
antecedent_broad = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
consequent_part = np.empty((NUMBER_OF_FUZZY_RULE, TEACHER_SIZE+1))
consequent = np.zeros(NUMBER_OF_FUZZY_RULE)

In [127]:
#まず前件部の値を初期化する
#ただし、ここは簡略型ファジィ推論と同様
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(NUMBER_OF_INPUT):
        if ((int)(i / (NUMBER_OF_FUZZY_PARTITION ** (NUMBER_OF_INPUT - (j+1))))) % NUMBER_OF_FUZZY_PARTITION == 0:
            antecedent_center[i,j] = 0
            antecedent_broad[i,j] = 1
        elif ((int)(i / (NUMBER_OF_FUZZY_PARTITION ** (NUMBER_OF_INPUT - (j+1))))) % NUMBER_OF_FUZZY_PARTITION == 1:
            antecedent_center[i,j] = 0.5
            antecedent_broad[i,j] = 0.5
        else:
            antecedent_center[i,j] = 1
            antecedent_broad[i,j] = 1

In [128]:
#次に後件部の値を初期化する
#後件部の値は簡略型とは違う
#教師データに使う73データと定数の74項から後件部実数値は構成される
#74番目の値は定数とする
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(TEACHER_SIZE+1):
        consequent_part[i,j] = 0.01
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(TEACHER_SIZE+1):
        if j == TEACHER_SIZE:
            consequent[i] = consequent[i] + consequent_part[i,j]
        else:
            consequent[i] = consequent[i] + (consequent_part[i,j] * y_teacher_data[j])
            #print(consequent_part[i,j] * y_teacher_data[j])
print(consequent)

[0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 

In [ ]:
#次に教師データを用いて学習を行う
#学習回数を設定する
TRAIN_TIME = 500
#学習係数を設定する
LEARNING_ANTECEDENT_BROAD = 0.001
LEARNING_ANTECEDENT_CENTER = 0.001
LEARNING_CONSEQUENT = 0.001
#まず、各ルールに基づいて入力変数をファジィ化（メンバシップ関数に代入）する
membership_function = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))

for time in range(TRAIN_TIME):
    for i in range(TEACHER_SIZE):
        delta_broad = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
        delta_center = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
        #適合度を初期化する
        adaptability = np.ones(NUMBER_OF_FUZZY_RULE)
        for j in range(NUMBER_OF_FUZZY_RULE):
            for k in range(NUMBER_OF_INPUT):
                if(x_teacher_data[i,k] >= antecedent_center[j,k] - antecedent_broad[j,k]) and x_teacher_data[i,k] <= antecedent_center[j,k]:
                    membership_function[j,k] = (x_teacher_data[i,k] - (antecedent_center[j,k] - antecedent_broad[j,k])) / antecedent_broad[j,k]
                    delta_broad[j,k] = (antecedent_center[j,k] - x_teacher_data[i,k]) / (antecedent_broad[j,k]**2)
                    delta_center[j,k] = -(1 / antecedent_broad[j,k])
                elif(x_teacher_data[i,k] > antecedent_center[j,k]) and (x_teacher_data[i,k] <= antecedent_center[j,k] + antecedent_broad[j,k]):
                    membership_function[j,k] = -(x_teacher_data[i,k] - (antecedent_center[j,k] + antecedent_broad[j,k])) / antecedent_broad[j,k]
                    delta_broad[j,k] = -(antecedent_center[j,k] - x_teacher_data[i,k]) / (antecedent_broad[j,k]**2)
                    delta_center[j,k] = 1 / antecedent_broad[j,k]
            #各ルールにおける適合度を求める
            for k in range(NUMBER_OF_INPUT):
                adaptability[j] = adaptability[j] * membership_function[j,k]
        #各データに対して予測結果を求める
        output = 0
        output = np.sum(np.dot(adaptability, consequent)) / np.sum(adaptability)
        if(time == TRAIN_TIME-1):
            print(output, y_teacher_data[i])
        #出力値を求めた次に後件部と前件部の更新を行う
        #まずは後件部の値の更新を行う
        #consequent_partの更新を行った後に、consequentに変換を行う
        for j in range(NUMBER_OF_FUZZY_RULE):
            for k in range(TEACHER_SIZE + 1):
                if k == TEACHER_SIZE:
                    consequent_part[j,k] = consequent_part[j,k] + (LEARNING_CONSEQUENT * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output)
                else:
                    consequent_part[j,k] = consequent_part[j,k] + (LEARNING_CONSEQUENT * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output) * y_teacher_data[k]
        #次に前件部の値の更新を行う
        for j in range(NUMBER_OF_FUZZY_RULE):
            for k in range(NUMBER_OF_INPUT):
                antecedent_center[j,k] = antecedent_center[j,k] + (LEARNING_ANTECEDENT_CENTER * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output) * (consequent[j] - output) * delta_center[j,k]
                antecedent_broad[j,k] = antecedent_broad[j,k] + (LEARNING_ANTECEDENT_BROAD * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output) * (consequent[j] - output) * delta_broad[j,k]